#### Problem Statement from Kaggle: 

### In this competition, you will be analyzing weather data and GIS data and predicting whether or not West Nile virus is present, for a given _time_, _location_, and _species_. 


In [4]:
import pandas as pd
import numpy as np
import geopy as gp
from geopy.geocoders import Nominatim
from geopy.distance import vincenty


In [5]:
# Verify geo locator is in place to assist with coordinates
geolocator = Nominatim()
location = geolocator.geocode("175 5th Avenue NYC")
print(location)

Flatiron Building, 175, 5th Avenue, Flatiron Building, Manhattan Community Board 5, New York County, NYC, New York, 10010, United States of America


#### Read in csv data files

In [7]:
train = pd.read_csv('../assets/train.csv')
#test = pd.read_csv('../assets/test.csv')
spray = pd.read_csv('../assets/spray.csv')
weather = pd.read_csv('../assets/weather.csv')

print(train.shape)
print(spray.shape)
print(weather.shape)

(10506, 12)
(14835, 4)
(2944, 22)


## EDA: Part I
#### Part I will focus on the initial review of data and joining data from the spray, training and weather datasets into one dataframe

### Spray Data
**Data for just 2011 and 2013**

In [8]:
spray.head()

,Date,Time,Latitude,Longitude
0,2011-08-29,6:56:58 PM,42.391623,-88.089163
1,2011-08-29,6:57:08 PM,42.391348,-88.089163
2,2011-08-29,6:57:18 PM,42.391022,-88.089157
3,2011-08-29,6:57:28 PM,42.390637,-88.089158
4,2011-08-29,6:57:38 PM,42.390410,-88.088858


In [9]:
spray.info()
# Missing Time values, but we can leave them

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14835 entries, 0 to 14834
Data columns (total 4 columns):
Date         14835 non-null object
Time         14251 non-null object
Latitude     14835 non-null float64
Longitude    14835 non-null float64
dtypes: float64(2), object(2)
memory usage: 463.7+ KB


In [383]:
# Fill missing Time values with empty
#spray['Time'].fillna('', inplace=True)

In [10]:
# Convert Date to datetime and break apart
spray['Date'] = pd.to_datetime(spray['Date'])
spray['Year'] = spray['Date'].apply(lambda x: x.year)
spray['Month'] = spray['Date'].apply(lambda x: x.month)
spray['Day'] = spray['Date'].apply(lambda x: x.day)

In [11]:
spray['Date'].value_counts()

2013-08-15    2668
2013-08-29    2302
2013-07-17    2202
2011-09-07    2114
2013-07-25    1607
2013-08-22    1587
2013-08-08    1195
2013-09-05     924
2013-08-16     141
2011-08-29      95
Name: Date, dtype: int64

#### Question: Are the spray coordinates unique for each observations or are the same coordinates being sprayed more than once?

** !!! ** There is some bad data. One location shows up 541 time for the same Date and Time and another location shows up twice. This is not correct. Same date might be okay, but same Date and Time is questionable.

#### Answer: The spray coordinates are unique.  The dataset does not show that locations were sprayed multiple times for 2011 and 2013.

In [12]:
df_spray = pd.DataFrame({'count' : spray.groupby(['Date','Latitude','Longitude','Time', 'Year', 'Month']).size()}).reset_index()
df_spray.sort_values('count', ascending=False).head(10)
# We will use this dataframe for spray info going forward since it removes the multiples issue 

,Date,Latitude,Longitude,Time,Year,Month,count
1012,2011-09-07,41.986460,-87.794225,7:44:32 PM,2011,9,541
970,2011-09-07,41.983917,-87.793088,7:43:40 PM,2011,9,2
0,2011-08-29,42.389460,-88.093895,7:11:28 PM,2011,8,1
9145,2013-08-22,41.726920,-87.603145,9:12:40 PM,2013,8,1
9135,2013-08-22,41.726590,-87.617912,10:45:46 PM,2013,8,1
9136,2013-08-22,41.726617,-87.622322,10:53:56 PM,2013,8,1
9137,2013-08-22,41.726663,-87.607023,10:22:46 PM,2013,8,1
9138,2013-08-22,41.726665,-87.613672,11:00:56 PM,2013,8,1
9139,2013-08-22,41.726682,-87.594917,8:49:50 PM,2013,8,1
9140,2013-08-22,41.726713,-87.616685,10:46:46 PM,2013,8,1


### Training Data for Mosquito testing

In [13]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10506 entries, 0 to 10505
Data columns (total 12 columns):
Date                      10506 non-null object
Address                   10506 non-null object
Species                   10506 non-null object
Block                     10506 non-null int64
Street                    10506 non-null object
Trap                      10506 non-null object
AddressNumberAndStreet    10506 non-null object
Latitude                  10506 non-null float64
Longitude                 10506 non-null float64
AddressAccuracy           10506 non-null int64
NumMosquitos              10506 non-null int64
WnvPresent                10506 non-null int64
dtypes: float64(2), int64(4), object(6)
memory usage: 985.0+ KB


In [14]:
train.describe()

,Block,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
count,10506.000000,10506.000000,10506.000000,10506.000000,10506.000000,10506.000000
mean,35.687797,41.841139,-87.699908,7.819532,12.853512,0.052446
std,24.339468,0.112742,0.096514,1.452921,16.133816,0.222936
min,10.000000,41.644612,-87.930995,3.000000,1.000000,0.000000
25%,12.000000,41.732984,-87.760070,8.000000,2.000000,0.000000
50%,33.000000,41.846283,-87.694991,8.000000,5.000000,0.000000
75%,52.000000,41.954690,-87.627796,9.000000,17.000000,0.000000
max,98.000000,42.017430,-87.531635,9.000000,50.000000,1.000000


In [15]:
# Convert date from object to datetime type and the extract parts into separate columns
train['Date'] = pd.to_datetime(train['Date'])
train['Year'] = train['Date'].apply(lambda x: x.year)
train['Month'] = train['Date'].apply(lambda x: x.month)
train['Day'] = train['Date'].apply(lambda x: x.day)

In [16]:
train['Species'].value_counts()

CULEX PIPIENS/RESTUANS    4752
CULEX RESTUANS            2740
CULEX PIPIENS             2699
CULEX TERRITANS            222
CULEX SALINARIUS            86
CULEX TARSALIS               6
CULEX ERRATICUS              1
Name: Species, dtype: int64

In [560]:
len(train['Trap'].value_counts())
#train['Trap'].value_counts()

136

### Join spray data to mosquito test data

The spray location coordinates will not match the trap coordinates exactly so we will find the nearest spray location. Note provided spray data is limited to 2013 and 2011 and the data for 2011 is very limited.  To find the nearest spray location for a trap... 
- Take the mosquito training year and date to determine appropriate spray data (spray observation must of the same year and also occur before the training data.  Ex. 08-29-2013 is not a valid spray date for a training date of 08-04-2013 since the spray occurs afterwards.  
- Find the distance from each spray location to the trap location
- Finally take the smallest distance and assume that spray location is the nearest to the trap

In [17]:
# Below is mask for determining if date is between range
# mask = (df['date'] > start_date) & (df['date'] <= end_date)

import datetime

def find_spray_datetime(x):
    location = [x.Latitude, x.Longitude]
    distances = []
    spray_info = []
    # get spray observations for given year AND date/time prior to mosquito test date
    df = pd.DataFrame(df_spray[(df_spray['Date'] < x.Date) & (df_spray['Year'] == x.Year)]).reset_index()
    # iterate through resulting set and grab distance in miles, date and lat and long as well
    if df.shape[0] > 0:
        for index, row in df.iterrows():
            dict = {}
            spray_location = [row.Latitude, row.Longitude]
            # get distance between trap and spray location
            distance = vincenty(location, spray_location).miles
            distance = str(round(distance, 6))
            distances.append(distance)
            dict['distance'] = distance
            dict['spray_info'] = '|'.join([row['Date'].strftime('%Y-%m-%d'), distance, str(row.Latitude), str(row.Longitude)])
            spray_info.append(dict)
            
        # sort distances
        distances.sort(reverse=False)
        #print(distances)
        shortest_distance = distances.pop(0)
        for d in spray_info:
            if shortest_distance == d['distance']:
                info = d['spray_info']
                break
        return info
    else:
        # return none if no valid spray info for mosquito test date
        return 'none'
            

In [18]:
## !! WARNING Cell will call find_spray_datetime which takes about 30 minutes for the whole dataset
# Only call the find spray time if year is 2011 or 2013 and month is after July.  
# Spray data does not exist for other times
train['spray_info'] = train[(train['Year'] >= 2011) & (train['Month'] >= 7)].apply(lambda x: find_spray_datetime(x), axis=1)

In [19]:
# Since we only having spraying data for 2011 and 2013, 
# not all training observations will have spray info
train['spray_info'].fillna('none', inplace=True) 

In [20]:
# Run cell to get a quick look at new column
# Of course, this column will need furthprocessing
# Spray info = spray date | distance from nearest spray location in miles | spray latitude | spray longitude

df_sprayinfo = train[['Date','Species', 'Trap','spray_info']][train['spray_info'] != 'none'] 
print(df_sprayinfo.shape)
df_sprayinfo.head()

(2195, 4)


,Date,Species,Trap,spray_info
7574,2011-09-01,CULEX PIPIENS/RESTUANS,T002,2011-08-29|33.495546|42.390395|-88.08831500000001
7575,2011-09-01,CULEX PIPIENS,T002,2011-08-29|33.495546|42.390395|-88.08831500000001
7576,2011-09-01,CULEX PIPIENS,T046,2011-08-29|41.037341|42.390395|-88.08831500000001
7577,2011-09-01,CULEX RESTUANS,T048,2011-08-29|42.447767|42.390395|-88.08831500000001
7578,2011-09-01,CULEX PIPIENS/RESTUANS,T049,2011-08-29|40.717225|42.390395|-88.08831500000001


** Process spray_info delimted string and break into separate columns **

In [21]:
# helper methods to deal with spray_info delimited string

def get_spray_date(x):
    values = x.split('|')
    return values[0]

def get_spray_distance(x):
    values = x.split('|')
    return values[1]

def get_spray_latitude(x):
    values = x.split('|')
    return clean_coordinate(values[2])

def get_spray_longitude(x):
    values = x.split('|')
    return clean_coordinate(values[3])

def clean_coordinate(coord):
    # split coord on decimal
    coord_parts = coord.split('.')
    # remove trailing numbers resulting from string conversion of float
    decimal_cleaned = coord_parts[1][0:6]
    return '.'.join([coord_parts[0], decimal_cleaned])
    

In [22]:
# code to test helper methods - can eventually remove
print(get_spray_latitude('2013-08-22|0.887871|41.7652766666667|-87.61852'))
print(get_spray_longitude('2013-08-22|0.887871|41.7652766666667|-87.618522222'))
print(get_spray_distance('2013-08-22|0.887871|41.7652766666667|-87.618522222'))

41.765276
-87.618522
0.887871


In [23]:
# Call helper methods to break up spray_info string. 
# Filter out 'none' values -> 'none' = no valid spray observations found for training date
train['SprayDate'] = train['spray_info'][train['spray_info'] != 'none'].apply(get_spray_date)
train['SprayDistance'] = train['spray_info'][train['spray_info'] != 'none'].apply(get_spray_distance)
train['SprayLatitude'] = train['spray_info'][train['spray_info'] != 'none'].apply(get_spray_latitude)
train['SprayLongitude'] = train['spray_info'][train['spray_info'] != 'none'].apply(get_spray_longitude)

In [24]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10506 entries, 0 to 10505
Data columns (total 20 columns):
Date                      10506 non-null datetime64[ns]
Address                   10506 non-null object
Species                   10506 non-null object
Block                     10506 non-null int64
Street                    10506 non-null object
Trap                      10506 non-null object
AddressNumberAndStreet    10506 non-null object
Latitude                  10506 non-null float64
Longitude                 10506 non-null float64
AddressAccuracy           10506 non-null int64
NumMosquitos              10506 non-null int64
WnvPresent                10506 non-null int64
Year                      10506 non-null int64
Month                     10506 non-null int64
Day                       10506 non-null int64
spray_info                10506 non-null object
SprayDate                 2195 non-null object
SprayDistance             2195 non-null object
SprayLatitude             

In [25]:
# Convert new columns to float
train['SprayDistance'] = train['SprayDistance'].astype('float')
train['SprayLatitude'] = train['SprayLatitude'].astype('float')
train['SprayLongitude'] = train['SprayLongitude'].astype('float')

In [28]:
# Run to verify that conversion took
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10506 entries, 0 to 10505
Data columns (total 20 columns):
Date                      10506 non-null datetime64[ns]
Address                   10506 non-null object
Species                   10506 non-null object
Block                     10506 non-null int64
Street                    10506 non-null object
Trap                      10506 non-null object
AddressNumberAndStreet    10506 non-null object
Latitude                  10506 non-null float64
Longitude                 10506 non-null float64
AddressAccuracy           10506 non-null int64
NumMosquitos              10506 non-null int64
WnvPresent                10506 non-null int64
Year                      10506 non-null int64
Month                     10506 non-null int64
Day                       10506 non-null int64
spray_info                10506 non-null object
SprayDate                 10506 non-null object
SprayDistance             10506 non-null float64
SprayLatitude          

In [27]:
# Replace nulls, again only a small set of training observations will include spray info
train['SprayDistance'].fillna(0.0, inplace=True)
train['SprayLatitude'].fillna(0.0, inplace=True)
train['SprayLongitude'].fillna(0.0, inplace=True)
train['SprayDate'].fillna('', inplace=True)

In [29]:
train[['Date', 'Block', 'Trap', 'SprayDate', 'SprayDistance', 'SprayLatitude', 'SprayLongitude', 'SprayDate']][train['SprayDistance'].between(0.0001, 3, inclusive=True)].head()

,Date,Block,Trap,SprayDate,SprayDistance,SprayLatitude,SprayLongitude,SprayDate
7686,2011-09-12,41,T002,2011-09-07,0.949832,41.968441,-87.801713,2011-09-07
7687,2011-09-12,41,T002,2011-09-07,0.949832,41.968441,-87.801713,2011-09-07
7688,2011-09-12,41,T002,2011-09-07,0.949832,41.968441,-87.801713,2011-09-07
7700,2011-09-12,36,T011,2011-09-07,1.952208,41.971056,-87.818440,2011-09-07
7724,2011-09-12,46,T003,2011-09-07,1.601834,41.969340,-87.787978,2011-09-07


** Group the training data by Year, Month, Trap and Species and determing if WNV is present for that group **

In [344]:
## REMOVE ?
## Believe this is no longer need
#df_train = pd.DataFrame(train.groupby(['Year','Month','Species', 'Block','Trap', 'Street','Latitude', 'Longitude']).agg({'NumMosquitos':'sum','WnvPresent': 'sum', 'Date': 'size'})).reset_index()

#df_train['WnvPresentForGroup'] = np.where(df_train['WnvPresent'] > 0, 1, 0)
#print(df_train.shape)
#df_train.columns = ['Year','Month','Species', 'Block','Trap', 'Street','Latitude', 'Longitude', 'SumNumMosquitos', 'SumWnvPresent', 'GroupCount', 'WnvPresentForGroup']
#df_train.sort_values(['Year','Month','Trap', 'WnvPresentForGroup'], ascending=False).head(5)

(3480, 12)


,Year,Month,Species,Block,Trap,Street,Latitude,Longitude,SumNumMosquitos,SumWnvPresent,GroupCount,WnvPresentForGroup
3310,2013,9,CULEX PIPIENS,10,T903,W OHARE,41.957799,-87.930995,20,0,1,0
3381,2013,9,CULEX PIPIENS/RESTUANS,10,T903,W OHARE,41.957799,-87.930995,10,0,1,0
3309,2013,9,CULEX PIPIENS,10,T900,W OHARE AIRPORT,41.974689,-87.890615,546,5,23,1
3380,2013,9,CULEX PIPIENS/RESTUANS,10,T900,W OHARE AIRPORT,41.974689,-87.890615,640,9,19,1
3451,2013,9,CULEX RESTUANS,10,T900,W OHARE AIRPORT,41.974689,-87.890615,45,1,3,1


In [54]:
#  KEEP - helpful reference to timedelta

#(df['date'] > start_date) & (df['date'] <= end_date)
#import datetime
#spray['DatePlus15'] = spray['Date'].apply(lambda x: x + datetime.timedelta(days=15))
#end_date = date_1 + datetime.timedelta(days=10)

### Weather Data

** _It is believed that hot and dry conditions are more favorable for West Nile virus than cold and wet. We provide you with the dataset from NOAA of the weather conditions of 2007 to 2014, during the months of the tests._** 

Based on the above comment from Kaggle, I will focus on **Heating** and **Cooling** days and **Total Precipitation**

Link to NOAA doc that explains heating and cooling days
http://www.cpc.ncep.noaa.gov/products/analysis_monitoring/cdus/degree_days/ddayexp.shtml

In [30]:
print(weather.shape)
weather.head()

(2944, 22)


,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,...,,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,...,,M,M,M,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,...,BR,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,M,42,47,13,0,...,BR HZ,M,M,M,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,...,,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9


In [31]:
weather.info() 
# No nulls, but will have to change column types

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2944 entries, 0 to 2943
Data columns (total 22 columns):
Station        2944 non-null int64
Date           2944 non-null object
Tmax           2944 non-null int64
Tmin           2944 non-null int64
Tavg           2944 non-null object
Depart         2944 non-null object
DewPoint       2944 non-null int64
WetBulb        2944 non-null object
Heat           2944 non-null object
Cool           2944 non-null object
Sunrise        2944 non-null object
Sunset         2944 non-null object
CodeSum        2944 non-null object
Depth          2944 non-null object
Water1         2944 non-null object
SnowFall       2944 non-null object
PrecipTotal    2944 non-null object
StnPressure    2944 non-null object
SeaLevel       2944 non-null object
ResultSpeed    2944 non-null float64
ResultDir      2944 non-null int64
AvgSpeed       2944 non-null object
dtypes: float64(1), int64(5), object(16)
memory usage: 506.1+ KB


In [32]:
# Drop rows that have an M (missing data)
df_weather = weather[(weather.Cool != 'M') & (weather.Heat != 'M') & (weather.PrecipTotal != 'M')]
print(df_weather.shape)

(2931, 22)


In [33]:
# Drop rows where precip == T
df_weather = df_weather[df_weather.PrecipTotal.str.replace(' ', '') != 'T']
print(df_weather.shape)

(2614, 22)


In [34]:
df_weather['Cool'] = df_weather['Cool'].astype('int')
df_weather['Heat'] = df_weather['Heat'].astype('int')
df_weather['PrecipTotal'] = df_weather['PrecipTotal'].astype('float')

In [38]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2614 entries, 0 to 2943
Data columns (total 22 columns):
Station        2614 non-null int64
Date           2614 non-null object
Tmax           2614 non-null int64
Tmin           2614 non-null int64
Tavg           2614 non-null object
Depart         2614 non-null object
DewPoint       2614 non-null int64
WetBulb        2614 non-null object
Heat           2614 non-null int64
Cool           2614 non-null int64
Sunrise        2614 non-null object
Sunset         2614 non-null object
CodeSum        2614 non-null object
Depth          2614 non-null object
Water1         2614 non-null object
SnowFall       2614 non-null object
PrecipTotal    2614 non-null float64
StnPressure    2614 non-null object
SeaLevel       2614 non-null object
ResultSpeed    2614 non-null float64
ResultDir      2614 non-null int64
AvgSpeed       2614 non-null object
dtypes: float64(2), int64(7), object(13)
memory usage: 469.7+ KB


** Considered just grabbing data for Station 1, but better to get average of Station data since some records are removed due to Missing and Trace values **

In [317]:
#df_weather1 = df_weather[['Date','Heat', 'Cool', 'PrecipTotal']][df_weather['Station'] == 1]
#df_weather1.head(31)

,Date,Heat,Cool,PrecipTotal
0,2007-05-01,0,2,0.00
2,2007-05-02,14,0,0.00
4,2007-05-03,9,0,0.00
10,2007-05-06,6,0,0.00
14,2007-05-08,0,3,0.00
16,2007-05-09,0,4,0.13
18,2007-05-10,0,5,0.00
20,2007-05-11,4,0,0.00
22,2007-05-12,10,0,0.00
24,2007-05-13,9,0,0.00


** Better to get average of Station 1 and Station 2 **

In [36]:
# Group by date so eah group includes data from Station and Station 2 and then get averages
# Note to remember that some obervations were dropped due Missing and Trace
df_weather2 = pd.DataFrame(df_weather.groupby(['Date']).agg({'Heat': 'mean', 'Cool': 'mean', 'PrecipTotal': 'mean'})).reset_index()

In [37]:
# convert date from object to datetime and extract components into separate columns
df_weather2['Date'] = pd.to_datetime(df_weather2['Date'])
df_weather2['Year'] = df_weather2['Date'].apply(lambda x: x.year)
df_weather2['Month'] = df_weather2['Date'].apply(lambda x: x.month)
df_weather2['Day'] = df_weather2['Date'].apply(lambda x: x.day)

In [39]:
df_weather2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1406 entries, 0 to 1405
Data columns (total 7 columns):
Date           1406 non-null datetime64[ns]
Heat           1406 non-null float64
Cool           1406 non-null float64
PrecipTotal    1406 non-null float64
Year           1406 non-null int64
Month          1406 non-null int64
Day            1406 non-null int64
dtypes: datetime64[ns](1), float64(3), int64(3)
memory usage: 77.0 KB


#### Get monthly averages

In [40]:
df_weather3 = pd.DataFrame(df_weather2.groupby(['Year', 'Month']).agg({'Heat': 'mean', 'Cool': 'mean', 'PrecipTotal': 'mean'})).reset_index()
print(df_weather3.shape)

# convert back to integers
df_weather3['Heat'] = round(df_weather3['Heat']).astype('int')
df_weather3['Cool'] = round(df_weather3['Cool']).astype('int')

(48, 5)


In [634]:
# Following will show us the year/month with highest precipitation
df_weather3.sort_values(['PrecipTotal'], ascending=False).head(10)

,Year,Month,Heat,Cool,PrecipTotal
10,2008,9,1,3,0.382333
20,2010,7,0,14,0.331897
3,2007,8,0,10,0.306667
45,2014,8,0,10,0.275645
43,2014,6,0,7,0.250345
26,2011,7,0,15,0.247500
19,2010,6,0,7,0.247500
13,2009,6,2,6,0.229821
24,2011,5,8,2,0.229815
17,2009,10,15,0,0.208548


In [41]:
df_weather3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 5 columns):
Year           48 non-null int64
Month          48 non-null int64
Heat           48 non-null int64
Cool           48 non-null int64
PrecipTotal    48 non-null float64
dtypes: float64(1), int64(4)
memory usage: 2.0 KB


### Join monthly weather averages to mosquito training data 

**The following method is a quick way to append month averages to the mosquito training data.
The three values in the delimited string will ultimately need to be in separate columns**

In [42]:
def add_weather_data(row):
    df = df_weather3[(df_weather3['Year'] == row['Year']) & (df_weather3['Month'] == row['Month'])]
    precip = str(round(df['PrecipTotal'].values[0], 4))
    heat = str(df['Heat'].values[0])
    cool = str(df['Cool'].values[0])
    return '|'.join([heat, cool, precip])

In [43]:
train['Weather_Info'] = train.apply(lambda x: add_weather_data(x), axis=1 )

In [44]:
# Take a quick look at the new column added to the training data
train[['Year', 'Month', 'Trap','Weather_Info']].head()

,Year,Month,Trap,Weather_Info
0,2007,5,T002,3|3|0.0636
1,2007,5,T002,3|3|0.0636
2,2007,5,T007,3|3|0.0636
3,2007,5,T015,3|3|0.0636
4,2007,5,T015,3|3|0.0636


In [45]:
# NOTE the following filter can be used to verify weather averages for a given year and month
df_weather3[(df_weather3['Year'] == 2007) & (df_weather3['Month'] == 5)]

,Year,Month,Heat,Cool,PrecipTotal
0,2007,5,3,3,0.063621


#### Process weather_info delimited string, break into separate columns

In [46]:
# Helper methods for breaking up weather_info delimited string
def get_heat(x):
    values = x.split('|')
    return values[0]

def get_cool(x):
    values = x.split('|')
    return values[1]

def get_precip(x):
    values = x.split('|')
    return values[2]

In [47]:
train['HeatMonthAvg'] = train['Weather_Info'].apply(get_heat)
train['CoolMonthAvg'] = train['Weather_Info'].apply(get_cool)
train['PrecipMonthAvg'] = train['Weather_Info'].apply(get_precip)

### Training data with spray and weather info added

In [48]:
train.columns

Index(['Date', 'Address', 'Species', 'Block', 'Street', 'Trap',
       'AddressNumberAndStreet', 'Latitude', 'Longitude', 'AddressAccuracy',
       'NumMosquitos', 'WnvPresent', 'Year', 'Month', 'Day', 'spray_info',
       'SprayDate', 'SprayDistance', 'SprayLatitude', 'SprayLongitude',
       'Weather_Info', 'HeatMonthAvg', 'CoolMonthAvg', 'PrecipMonthAvg'],
      dtype='object')

#### The following two filters show different views the training data. The first focuses on weather and the second more on spraying information.  Note spraying info is sparse and only exists for a small number of training observations, most in 2013.

In [49]:
train[['Date','Species','Block','Trap',
       'WnvPresent','NumMosquitos','HeatMonthAvg',
       'CoolMonthAvg','PrecipMonthAvg', 'SprayDate']].tail(20)

,Date,Species,Block,Trap,WnvPresent,NumMosquitos,HeatMonthAvg,CoolMonthAvg,PrecipMonthAvg,SprayDate
10486,2013-09-26,CULEX PIPIENS,10,T900,0,16,2,5,0.0783,2013-07-17
10487,2013-09-26,CULEX PIPIENS,10,T900,0,9,2,5,0.0783,2013-07-17
10488,2013-09-26,CULEX PIPIENS,10,T900,0,11,2,5,0.0783,2013-07-17
10489,2013-09-26,CULEX PIPIENS,10,T900,0,1,2,5,0.0783,2013-07-17
10490,2013-09-26,CULEX PIPIENS/RESTUANS,48,T222,0,1,2,5,0.0783,2013-08-15
10491,2013-09-26,CULEX PIPIENS/RESTUANS,51,T223,0,11,2,5,0.0783,2013-08-15
10492,2013-09-26,CULEX PIPIENS,51,T223,0,1,2,5,0.0783,2013-08-15
10493,2013-09-26,CULEX PIPIENS/RESTUANS,82,T225,0,3,2,5,0.0783,2013-08-29
10494,2013-09-26,CULEX PIPIENS/RESTUANS,65,T227,0,7,2,5,0.0783,2013-08-22
10495,2013-09-26,CULEX PIPIENS,65,T227,0,1,2,5,0.0783,2013-08-22


In [50]:
# Note spray data is filter so that we only see rows 
# where the distance is 3 miles or less from the spray location
train[['Date','Species','Block','Trap',
       'NumMosquitos','WnvPresent',
       'SprayDate','SprayDistance',
       'SprayLatitude','SprayLongitude']][train['SprayDistance'].between(0.0001, 3.0, inclusive=True)]

,Date,Species,Block,Trap,NumMosquitos,WnvPresent,SprayDate,SprayDistance,SprayLatitude,SprayLongitude
7686,2011-09-12,CULEX PIPIENS/RESTUANS,41,T002,4,0,2011-09-07,0.949832,41.968441,-87.801713
7687,2011-09-12,CULEX RESTUANS,41,T002,3,0,2011-09-07,0.949832,41.968441,-87.801713
7688,2011-09-12,CULEX PIPIENS,41,T002,1,0,2011-09-07,0.949832,41.968441,-87.801713
7700,2011-09-12,CULEX PIPIENS/RESTUANS,36,T011,2,0,2011-09-07,1.952208,41.971056,-87.818440
7724,2011-09-12,CULEX PIPIENS/RESTUANS,46,T003,22,1,2011-09-07,1.601834,41.969340,-87.787978
7725,2011-09-12,CULEX RESTUANS,46,T003,5,0,2011-09-07,1.601834,41.969340,-87.787978
7726,2011-09-12,CULEX PIPIENS,46,T003,1,0,2011-09-07,1.601834,41.969340,-87.787978
7727,2011-09-12,CULEX PIPIENS/RESTUANS,70,T008,1,0,2011-09-07,1.726143,41.986460,-87.794225
7728,2011-09-12,CULEX PIPIENS/RESTUANS,61,T012,3,0,2011-09-07,2.198068,41.982766,-87.788191
7784,2011-09-12,CULEX PIPIENS/RESTUANS,10,T900,3,0,2011-09-07,2.784849,41.975748,-87.836559


In [52]:
# !! NOTE write to csv file
train.to_csv('../output/train_spray_weather.csv')

In [53]:
train_demo = pd.read_csv('../output/train_spray_weather.csv')
train_demo.head()
train_demo.columns

,Unnamed: 0,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,...,Day,spray_info,SprayDate,SprayDistance,SprayLatitude,SprayLongitude,Weather_Info,HeatMonthAvg,CoolMonthAvg,PrecipMonthAvg
0,0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,...,29,none,NaN,0.0,0.0,0.0,3|3|0.0636,3,3,0.0636
1,1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,...,29,none,NaN,0.0,0.0,0.0,3|3|0.0636,3,3,0.0636
2,2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,...,29,none,NaN,0.0,0.0,0.0,3|3|0.0636,3,3,0.0636
3,3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,...,29,none,NaN,0.0,0.0,0.0,3|3|0.0636,3,3,0.0636
4,4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,...,29,none,NaN,0.0,0.0,0.0,3|3|0.0636,3,3,0.0636


In [54]:
train_demo.columns

Index(['Unnamed: 0', 'Date', 'Address', 'Species', 'Block', 'Street', 'Trap',
       'AddressNumberAndStreet', 'Latitude', 'Longitude', 'AddressAccuracy',
       'NumMosquitos', 'WnvPresent', 'Year', 'Month', 'Day', 'spray_info',
       'SprayDate', 'SprayDistance', 'SprayLatitude', 'SprayLongitude',
       'Weather_Info', 'HeatMonthAvg', 'CoolMonthAvg', 'PrecipMonthAvg'],
      dtype='object')

###  Things to keep in mind (Discuss with team)
- Inflated zeros when dealing with spray info 
- With average heat/cool we can see things like 3/3 split - which means for month avg cooling was 62 and avg heating was 68